In [ ]:
import boto3
import pandas as pd
resource_client = boto3.client('resourcegroupstaggingapi')
sm_client = boto3.client('sagemaker')

In [ ]:
response = resource_client.get_resources(
    TagFilters=[
        {
            'Key': 'model',
            'Values': [
                'free_trial_xgboost', 'free_trial_sklearn_featurizer'
            ]
        },
    ],
    ResourcesPerPage=100,
    ResourceTypeFilters=['sagemaker:model']    
    )

In [ ]:
training_rows = []

for r in response['ResourceTagMappingList']:
    model_name = r['ResourceARN'].split('/')[1]
    response = sm_client.describe_model(
        ModelName=model_name
    )
    
    model_name = response['ModelName']
    image = response['PrimaryContainer']['Image']
    model_data_url = response['PrimaryContainer']['ModelDataUrl']
    role = response['ExecutionRoleArn']
    creation_time = response['CreationTime'].strftime("%Y-%m-%d %H:%M:%S")
    arn = response['ModelArn']
    
    row = [model_name, creation_time, image, model_data_url, role, arn]
    training_rows.append(row)

df = pd.DataFrame(training_rows, columns=['model_name', 'creation_time', 'image', 'model_data_url', 'role', 'arn'])

In [ ]:
df